# Web Scraping

In [1]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
print("Imported Libraries")

Imported Libraries


## APIs

In [2]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass

def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


## TASK 1: Request the Falcon9 Launch Wiki page from its URL

In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
web_data = requests.get(static_url)
soup = BeautifulSoup(web_data.content, "html.parser")
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

## TASK 2: Extract all column/variable names from the HTML table header

In [4]:
html_tables = soup.find_all('table')
first_launch_table = html_tables[2]
column_names = []
for row in first_launch_table.find_all('th'):
    column_name = extract_column_from_header(row)
    if column_name != None and len(column_name) > 0 and column_name != 'Date and time ( )':
        column_names.append(column_name)
print(column_names)

['Flight No.', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables

In [5]:
launch_dict= dict.fromkeys(column_names)

launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [6]:
for table_number, table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):

    for rows in table.find_all("tr"):

        row = rows.find_all('td')
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False

        if flag:

            launch_dict['Flight No.'].append(flight_number)
            
            datatimelist=date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)
              
            bv = booster_version(row[1])
            if not(bv):
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)
            
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)

            payload = row[3].a.string
            launch_dict['Payload'].append(payload)

            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)

            if row[6].a != None:
                customer = row[6].a.string
            else:
                customer = None
            launch_dict['Customer'].append(customer)

            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)

            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

## Verify Data

In [7]:
df = pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [8]:
df = df.replace(r'\r+|\n+|\t+','', regex=True)

In [9]:
with pd.option_context('display.max_rows', 15,
                       'display.max_columns', None,
                       'display.precision', 5,
                       ):
    print(df)

    Flight No. Launch site                               Payload Payload mass  \
0            1       CCAFS  Dragon Spacecraft Qualification Unit            0   
1            2       CCAFS                                Dragon            0   
2            3       CCAFS                                Dragon       525 kg   
3            4       CCAFS                          SpaceX CRS-1     4,700 kg   
4            5       CCAFS                          SpaceX CRS-2     4,877 kg   
..         ...         ...                                   ...          ...   
116        117       CCSFS                              Starlink    15,600 kg   
117        118         KSC                              Starlink   ~14,000 kg   
118        119       CCSFS                              Starlink    15,600 kg   
119        120         KSC                         SpaceX CRS-22     3,328 kg   
120        121       CCSFS                                 SXM-8     7,000 kg   

    Orbit   Customer Launch

## Save Data to CSV File

In [10]:
df.to_csv('spacex_web_scraped.csv', index=False)